In [ ]:
# Required packages
%pip install pandas requests python-dotenv httpx aiofiles dataclasses tqdm pyarrow clickhouse-connect clickhouse-driver

# External imports
import pandas as pd 
from dataclasses import dataclass
import json
from datetime import datetime
from tqdm import tqdm
import asyncio
import os
import ipaddress
from clickhouse_connect import get_client

# Internal imports
import config
import util
import objects

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ImportError: cannot import name 'Client' from 'clickhouse_connect' (/home/ubombar/Projects/Doctoral/iris-data-analysis/.venv/lib/python3.11/site-packages/clickhouse_connect/__init__.py)

In [2]:
cfg = config.Config()
cfg.response_format = "CSVWithNames"
qc = util.QueryClient(cfg)

measurements_table_df = pd.read_feather("../data/measurements/measurement-2024-11-18 22:44:16.feather")
selected_measurement_uuid = "007046a9_518e_46cb_8e70_e598b8bce831"
selected_measurements_df = measurements_table_df[
    (measurements_table_df['cleaned'] == True) & 
    (measurements_table_df['measurement_uuid'] == selected_measurement_uuid) & 
    (measurements_table_df['type'] == 'results')
]

selected_measurements_df['table_name'].unique()

array(['cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604',
       'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__4f6137c8_dfc5_4043_8005_d617407b7be3',
       'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__51fb56f4_5217_4108_a212_2d05806b16dc',
       'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__5bdab212_3d82_4eca_8c5a_39e1c8ded842',
       'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__70ce196e_9cb7_4467_b1a8_c40400f6f5df',
       'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__91b015da_729c_40cd_b051_d6439ce5c00d',
       'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__a05d071c_159d_448e_a349_95431bf96a51',
       'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__bc17c7f9_9d02_48de_b766_8c927c8c10fd',
       'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__ccdf7970_8877_42cc_960d_41d0d0d0c302',
       'cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__f39c7a18_5b00_4e57_

In [5]:
q = '''
    WITH table as (SELECT probe_dst_addr, reply_src_addr  FROM cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604 LIMIT 100)
    SELECT DISTINCT table.probe_dst_addr, table.reply_src_addr
    FROM table
'''

r = await qc.async_query_getall_str(q, format_it='df')
r

,probe_dst_addr,reply_src_addr
0,::ffff:1.0.213.11,::ffff:203.190.250.176
1,::ffff:1.0.213.11,::ffff:203.190.251.159
2,::ffff:1.0.213.11,::ffff:203.113.59.131
3,::ffff:1.0.213.11,::ffff:125.24.217.162
4,::ffff:1.0.213.11,::ffff:203.113.44.217
5,::ffff:1.0.213.11,::ffff:1.0.213.11
6,::ffff:1.0.213.39,::ffff:203.190.250.176
7,::ffff:1.0.213.39,::ffff:203.190.251.159
8,::ffff:1.0.213.39,::ffff:203.113.59.131
9,::ffff:1.0.213.39,::ffff:125.24.217.162


In [23]:
q = '''
SHOW CREATE TABLE cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604
'''

    # (SELECT probe_dst_addr, reply_src_addr FROM cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604)

r = await qc.async_query_getall_str(q, format_it='df')
print(r['statement'].iloc[0])

CREATE TABLE iris.cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604
(
    `capture_timestamp` DateTime,
    `probe_protocol` UInt8,
    `probe_src_addr` IPv6,
    `probe_dst_addr` IPv6,
    `probe_src_port` UInt16,
    `probe_dst_port` UInt16,
    `probe_ttl` UInt8,
    `quoted_ttl` UInt8,
    `reply_src_addr` IPv6,
    `reply_protocol` UInt8,
    `reply_icmp_type` UInt8,
    `reply_icmp_code` UInt8,
    `reply_ttl` UInt8,
    `reply_size` UInt16,
    `reply_mpls_labels` Array(Tuple(UInt32, UInt8, UInt8, UInt8)),
    `rtt` UInt16,
    `round` UInt8,
    `probe_dst_prefix` IPv6,
    `reply_src_prefix` IPv6,
    `private_probe_dst_prefix` UInt8,
    `private_reply_src_addr` UInt8,
    `destination_host_reply` UInt8,
    `destination_prefix_reply` UInt8,
    `valid_probe_protocol` UInt8,
    `time_exceeded_reply` UInt8
)
ENGINE = MergeTree
ORDER BY (probe_protocol, probe_src_addr, probe_dst_prefix, probe_dst_addr, probe_src_port, probe_dst_port, p

In [22]:
q = '''
SHOW CREATE DATABASE iris
'''

    # (SELECT probe_dst_addr, reply_src_addr FROM cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604)

r = await qc.async_query_getall_str(q, format_it='df')
r['statement'].iloc[0]

'CREATE DATABASE iris\nENGINE = Atomic'

In [26]:

async def run():
    q = 'SELECT * FROM cleaned_results__007046a9_518e_46cb_8e70_e598b8bce831__400a3c9b_57ed_4315_9489_917e601f3604 LIMIT 10'
    qc.cfg.response_format = "CSV"
    async for line in qc.async_query_lines(q):
        print(line)
    
await run()

"2024-05-22 02:26:07",1,"::ffff:10.0.0.2","::ffff:1.0.213.11",24000,0,4,1,"::ffff:203.190.250.176",1,11,0,252,96,"[]",563,1,"::ffff:1.0.213.0","::ffff:203.190.250.0",0,0,0,0,1,1
"2024-05-22 02:26:07",1,"::ffff:10.0.0.2","::ffff:1.0.213.11",24000,0,5,1,"::ffff:203.190.251.159",1,11,0,251,96,"[]",560,1,"::ffff:1.0.213.0","::ffff:203.190.251.0",0,0,0,0,1,1
"2024-05-22 02:26:07",1,"::ffff:10.0.0.2","::ffff:1.0.213.11",24000,0,6,1,"::ffff:203.113.59.131",1,11,0,246,168,"[(24029,3,1,1)]",646,1,"::ffff:1.0.213.0","::ffff:203.113.59.0",0,0,0,0,1,1
"2024-05-22 02:26:07",1,"::ffff:10.0.0.2","::ffff:1.0.213.11",24000,0,7,2,"::ffff:125.24.217.162",1,11,0,247,168,"[(24015,3,1,1)]",636,1,"::ffff:1.0.213.0","::ffff:125.24.217.0",0,0,0,0,1,1
"2024-05-22 02:26:07",1,"::ffff:10.0.0.2","::ffff:1.0.213.11",24000,0,8,1,"::ffff:203.113.44.217",1,11,0,248,96,"[]",644,1,"::ffff:1.0.213.0","::ffff:203.113.44.0",0,0,0,0,1,1
"2024-05-22 02:26:07",1,"::ffff:10.0.0.2","::ffff:1.0.213.11",24000,0,10,0,"::ffff:1.0.2